In [1]:
import pandas as pd
import numpy as np
import matplotlib as  plt

%matplotlib inline
pd.set_option('display.max_columns', 50)
from urllib.request import urlopen
from bs4  import BeautifulSoup

import re
import requests


baseurl  = 'https://www.daraz.pk/'

headers = {
    'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/74.0.3729.169 Safari/537.36',
}

r = requests.get('https://www.daraz.pk/')
soup = BeautifulSoup(r.content,'lxml')
# print(soup)

categories_sidebar_class ="lzd-site-menu-root-item"


# FINDING ALL THE CATEGORY OFFERED BY DARAZ
# categorylist = soup.find_all("li", class_ = "lzd-site-menu-root-item" )
categoryList = soup.select('li.lzd-site-menu-root-item > a > span')
# print(categoryList)

# FINDING ALL THE SUBCATEGORIES IN HTML
# EVERY SUBCATEGORY HAS DATA ATTRIBUTE --> cate_data 
subCatList  = soup.select('li.lzd-site-menu-sub-item > a ')

# MAKING THE LIST OF ALL THE LINK PRESENT IN SUBCATEGORIES
# NOT USED
links = []
for a in subCatList:
    links.append(f"https:{a['href']}")

# NOT USED
for index, val in enumerate(categoryList):
    subCategorieslist = soup.findAll(attrs={"class" : "lzd-site-menu-sub-item", "data-cate": f"cate_{index}_1"})

# for index,subCat in enumerate(subCategorieslist): 
#     categories[index]["subcategories"] =  subCat
# print(subCategorieslist)

# print(categoryslugs)




In [3]:
# MAKING THE MASTER LIST FROM THE CATEGORY LIST
masterCatList = []
for index, val in enumerate(categoryList):
    # USING REGEX TO FIND ALL THE ELEMENTS OF SUB CAT
    subCategorieslist = soup.findAll(attrs={"class" : "lzd-site-menu-sub-item", "data-cate": re.compile(f"^cate_{index + 1}")})
    for x in subCategorieslist:
        v = x.find('a').getText().strip() # GETTING SUBCATEGORY 
        l =  x.find('a')['href'] # GETTING LINK
        obj = {
            "category": val.getText(),
            "subCategory": v,
            "link": f"https:{l}"
        }
        masterCatList.append(obj)



In [3]:
import json


def getJsonData(link):
    r = requests.get(link)
    soup = BeautifulSoup(r.content,'lxml')
    s = ''
    for script in soup.select('script'):
        if 'window.pageData=' in script.text:
            s = script.text.replace('window.pageData=','')
#             break
            json_data = json.loads(s)
            return json_data

    
# NOT USED
def totalPage(jsonData):
#     listItems = jsoData['mods']['listItems']
    pageSize = int(jsonData['mainInfo']['pageSize'])
    totalResults = int(jsonData['mainInfo']['totalResults'])
    totalNumPage = int(round(totalResults/pageSize))
    return totalNumPage
                       

# NOT USED
def generateSubCatObj(linksList):
    subCatList = [{
    "link": "",
    "totalPage": 0
    }]
    for url in linksList:
        data = getJsonData(url) # sssss
        totPage = int(totalPage(data))
        subCatList.append({
            "link": url,
            "totalPage": totPage
        })
    return subCatList
                       
# NOT USED       
def generateCompleteData(obj):
    d = []
    for ind in obj:
        for num in range(0, ind.get("totalPage")):
            try:
                url = f"{ind.link}?page={num}"
                data = getJsonData(url)
                print(data)
                d.append(data)
            except:
                continue           
    return d  
    

# compData
           
                       
# for urls in links:
#      print(urls)
        
#     response = requests.request("GET", urls, headers=headers, data=payload)
    
#     y = json.loads(response)

In [5]:
# GETTING ALL THE DATA
def fetchCompleteData(mList):
    l = []
    count = 0
    for obj in mList:
        link = obj['link']
        jsonData = getJsonData(link)
        if jsonData:
            data = jsonData['mods']['listItems']
            pageSize = int(jsonData['mainInfo']['pageSize'])
            totalResults = int(jsonData['mainInfo']['totalResults'])
            totalNumPage = int(round(totalResults/pageSize))
            for num in range(0, totalNumPage):
                url = f"{link}?page={num}"
                jsonDataPerPage = getJsonData(url)
                dataPerPage = jsonData['mods']['listItems']
                for d in dataPerPage:
                    count += 1
                    name = d['name']
                    price = float(d['price'])
#             originalPrice = float(d['originalPrice'])
                    brandName = d['brandName']
                    productObject = {
                        "name": name,
                        "price": price,
#                 "originalPrice": originalPrice,
                        "brandName": brandName,
                        "category": obj["category"],
                        "subcategory": obj['subCategory']
                    }
                    l.append(productObject)
            
    return l

a = fetchCompleteData(masterCatList)


In [6]:
# MAKING DATAFRAME
table = pd.DataFrame(a)

In [6]:
table.to_excel("darazProductInfo.xlsx", index=False)